# Group 5 Assignment 4_inst 3

### Beiyuan Liu  1823772
### Yang Zhang  1726414
### Yewen Zhou 1821891

In [1]:
# Import all packages
import docplex
from docplex.mp.model import Model
import pandas as pd
import numpy as np

In [2]:
####### choose to run which instance #######
case = 3

# Deterministic Model

In [3]:
model = Model(name = "nightmare")

In [4]:
# Import data from excel
data = pd.ExcelFile('Assignment_4_Data_Surgery_scheduling.xlsx')
raw_data = data.parse('data')
costs = data.parse('Costs(in $ per hour)')
instances = data.parse('Sequencing Instances')

C:\Users\zyw\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


## Input Parameters

In [5]:
total_surgeries = instances.iloc[case,2]
surgeries = range(1,total_surgeries+1)
order = range(1,total_surgeries+1)
room = [1,2]
bigM = 100000

# shift time
T = instances.iloc[case,1]

# costs
cv = costs.iloc[1,0]
cw = costs.iloc[1,1]
co = costs.iloc[1,2]

In [6]:
# rename each surgery with a number
dic = {'A': 1, 'B': 2, 'C': 3,'D': 4,'E': 5,'F': 6,'G': 7,'H': 8,'I': 9,'J': 10}

In [7]:
# replace the letter of surgery in instance with its corresponding number
inst = np.zeros(total_surgeries)
for i in surgeries:
    inst[i-1] = dic[instances.iloc[case,i+2]]
inst = inst.astype(int)

In [8]:
# deterministic case
# store the surgery time
tp = np.zeros(10)
for i in range(1,11):
    tp[i-1] = raw_data.iloc[0,i]

ts = np.zeros(10)
for i in range(1,11):
    ts[i-1] = raw_data.iloc[1,i]

tc = np.zeros(10)
for i in range(1,11):
    tc[i-1] = raw_data.iloc[2,i]

## Define Decision Variables

In [9]:
# start time of surgery with order k in room r
S = model.continuous_var_matrix(room, order, name = "surgery_start_time")
# binary, whether surgery i is operated in room r with order k
X = model.binary_var_cube(surgeries, room, order, name = "whether_surgery")
# vacant time of urgery with order k in room r
V = model.continuous_var_matrix(room, order, name = "idle_time")
# binary, whether there is overtime at room r
O = model.binary_var_dict(room, name = "whether_overtime")
# overtime at room r
OT = model.continuous_var_dict(room, name = "overtime")
# empty time of room r
# empty time occurs only when the last surgery in the room is done before the shift time T
# in this case, the time window between the finishing time of last surgery and the shift end is the empty time
E = model.continuous_var_dict(room, name = "empty_time")

## Constraints

In [10]:
# each order positon can only be taken by one surgery, no matter in which room
for k in order:
    model.add_constraint(sum(sum(X[i,r,k] for i in surgeries) for r in room) == 1)

# each surgery can only be operated once in one room
for i in surgeries:
    model.add_constraint(sum(sum(X[i,r,k] for k in order) for r in room) == 1)

# if a surgery i doesn't take place in room r with order k, then the corresponding S[r,k] must be zero
# if a surgery i take place in room r with order k, then the corresponding surgery start time counts
# non-gegativity
for r in room:
    for k in order:
        model.add_constraint(S[r,k] <= sum(X[i,r,k]*bigM for i in surgeries))
        model.add_constraint(S[r,k] >= 0)

# let the first surgery be operated in room 1
model.add_constraint(sum(X[i,1,1] for i in surgeries) == 1)
model.add_constraint(S[1,1] - sum(X[i,1,1]*tp[inst[i-1]-1] for i in surgeries) >= 0)
model.add_constraint(S[2,1] == 0)

# the first surgery has no idle time and is operated directly after the preparation is done
# therefore the start time of first surgery equals to its corresponding preparation time
model.add_constraint(sum(S[r,1] for r in room) == sum(sum(X[i,r,1]*tp[inst[i-1]-1] for r in room) for i in surgeries))

# the surgery starts only after the last surgery is done
# therefore the start time of surgery is larger than the start time of last surgery plus its surgery time
for k in range(1,total_surgeries):
    model.add_constraint(sum(S[r,k] + sum(X[i,r,k]*ts[inst[i-1]-1] for i in surgeries) for r in room) <= 
                         sum(S[r,k+1] for r in room))

# idle time in room r with order k exists only when there is a surgery planned
# if there is no surgery, then the idle time is zero
# non-negativity
for r in room:
    for k in order:
        model.add_constraint(V[r,k] <= sum(X[i,r,k]*bigM for i in surgeries))
        model.add_constraint(V[r,k] >= 0)

# the first surgery has no idle time 
model.add_constraint(V[1,1] + V[2,1] == 0)

for r in room:
    # identify whether overtime in each room exists
    # if the time after clean-up of last surgery exceeds the shift length, then overtime exists and O[r] = 1, if not, then O[r] = 0
    model.add_constraint(O[r]*bigM 
                         >= sum(sum(X[i,r,k]*(tp[inst[i-1]-1] + ts[inst[i-1]-1] + tc[inst[i-1]-1]) for i in surgeries) for k in order) 
                             + sum(V[r,k] for k in order) - T*60)
    
    # relate empty time and overtime, empty time and overtime can't coexist in the same room
    model.add_constraint(E[r] <= (1-O[r])*bigM)
    
    # equation of the total used time in each room
    # for each room: time needed for all surgeries performed + all idle time + possible empty time = the shift length + possible overtime
    model.add_constraint(sum(sum(X[i,r,k]*(tp[inst[i-1]-1] + ts[inst[i-1]-1] + tc[inst[i-1]-1]) for i in surgeries) for k in order) 
                         + sum(V[r,k] for k in order) + E[r] == OT[r] + T*60)
    
    # relate the OT[r] and O[r], if overtime in one room doesn't exist, then OT[r] = 0
    model.add_constraint(OT[r] <= O[r]*bigM)
    
    # non-negativity
    model.add_constraint(OT[r] >= 0)
    model.add_constraint(E[r] >= 0)

# give idle time a lower bound
# the idle time of each surgery >= the start time of the surgery 
#                                 - its corresponding preparation time  - the used time of all surgeries ahead 
#                                 - all possible idle time of surgeries ahead
for r in room:
    for k in range(1,total_surgeries):
        model.add_constraint(V[r,k+1] >= S[r,k+1] - sum(X[i,r,k+1]*tp[inst[i-1]-1] for i in surgeries) 
                             - sum(sum(X[i,r,k1]*(tp[inst[i-1]-1] + ts[inst[i-1]-1] + tc[inst[i-1]-1]) for i in surgeries) for k1 in range(1,k+1))
                             - sum(V[r,k2] for k2 in range(1,k+1))
                            )
        
# consider all possible combinations in which two surgeries are performed consecutively in one room, no matter whether there is one or two 
# or three or more surgeries are performed between them in another room.

# if there are totally 5 surgeries, the combinations of two consecutively performed surgeries could be 1&2, 1&3, 1&4, 1&5, 2&3, 2&4, 2&5, 3&4, 3&5, 4&5
# if one of the combinations is ture (two surgeries are performed consecutively in one room), then LHS is the time after clean-up of a surgery 
# and the RHS is the start time of the following surgery - its corresponding preparation time - its possible idle time, which is exactly the time after clean-up of last surgery
# this also holds for combinations in which two surgeries are in the same room but not consecutively.

# if two surgeries of one combination is not in the same room, then the constraint will be either the time after clean-up of a surgery <= bigM
# or 0 <= a certain time + bigM. 
for r in room:
    for k in range(1,total_surgeries):
        for k1 in range(k,total_surgeries):
            model.add_constraint(S[r,k] + sum(X[i,r,k]*(ts[inst[i-1]-1] + tc[inst[i-1]-1]) for i in surgeries) <= 
                                 S[r,k1+1] - sum(X[i,r,k1+1]*tp[inst[i-1]-1] for i in surgeries) - V[r,k1+1] 
                                 + (1-sum(X[i,r,k1+1] for i in surgeries))*bigM
                                )
                                

## Objective Function

In [11]:
#surgeon waiting time
waiting_time = sum(S[r,k+1] - S[r,k] - sum(X[i,r,k]*ts[inst[i-1]-1] for i in surgeries) for r in room for k in range(1,total_surgeries))

#vacant time = idle time + empty time
vacant_time = sum(sum(V[r,k] for r in room) for k in order)

#overtime
overtime = sum(OT[r] for r in room)

#total_time = waiting_time + vacant_time + overtime
total_cost = cw/60*waiting_time + cv/60*vacant_time + co/60*overtime

model.minimize(total_cost)

## Solve the model

In [12]:
model.solve()
model.report()
model.print_solution()
print("The number of constraints is", model.number_of_constraints)
print("The CPU time is", model.solve_details.time, "seconds") # print the CPU time for the solve process

* model nightmare solved with objective = 982.567
objective: 982.567
  surgery_start_time_1_1=5.025
  surgery_start_time_1_3=50.708
  surgery_start_time_1_5=149.995
  surgery_start_time_1_6=0.000
  surgery_start_time_1_7=275.396
  surgery_start_time_2_2=18.066
  surgery_start_time_2_4=100.847
  surgery_start_time_2_6=193.935
  whether_surgery_1_1_1=1
  whether_surgery_1_2_3=-0
  whether_surgery_2_2_6=1
  whether_surgery_3_2_4=1
  whether_surgery_4_1_7=1
  whether_surgery_5_1_3=1
  whether_surgery_6_1_5=1
  whether_surgery_7_1_1=-0
  whether_surgery_7_2_2=1
  whether_surgery_7_2_3=0
  idle_time_1_7=11.072
  idle_time_2_4=5.475
  empty_time_1=36.240
  empty_time_2=164.248
The number of constraints is 147
The CPU time is 0.578999999910593 seconds


## Show the optimal sequence and OR assignment

In [13]:
slist = instances.iloc[case,3:total_surgeries+3].tolist()

# show optimal sequence
res_order = []
for k in order:
    for i in surgeries:
        for r in room:
            if X[i,r,k].solution_value>0.5:
                res_order.append(slist[i-1])
                
# show OR assignment
res_order_1 = []
res_order_2 = []
for k in order:
    for i in surgeries:
        if X[i,1,k].solution_value>0.5:
            res_order_1.append(slist[i-1])
        if X[i,2,k].solution_value>0.5:
            res_order_2.append(slist[i-1])

print("The optimal sequence should be:",res_order)
print("OR1's schedule (in order) should be:",res_order_1)
print("OR2's schedule (in order) should be:",res_order_2)

The optimal sequence should be: ['A', 'J', 'G', 'D', 'G', 'B', 'E']
OR1's schedule (in order) should be: ['A', 'G', 'G', 'E']
OR2's schedule (in order) should be: ['J', 'D', 'B']


# Worst Case Situation

In [14]:
model = Model(name = "nightmare_worst")

## Input Parameters

In [15]:
# Import worst case data from excel
worst_case_data = data.parse('Data_worst_case')

In [16]:
tp = np.zeros(10)
for i in range(1,11):
    tp[i-1] = worst_case_data.iloc[0,i]

ts = np.zeros(10)
for i in range(1,11):
    ts[i-1] = worst_case_data.iloc[1,i]

tc = np.zeros(10)
for i in range(1,11):
    tc[i-1] = worst_case_data.iloc[2,i]

## Define Desicion Variables

In [17]:
#start time of surgery
S = model.continuous_var_matrix(room, order, name = "surgery_start_time")
#binary
X = model.binary_var_cube(surgeries, room, order, name = "whether_surgery")
#vacant time
V = model.continuous_var_matrix(room, order, name = "idle_time")
#binary, whether overtime at room r
O = model.binary_var_dict(room, name = "whether_overtime")
#overtime at room r
OT = model.continuous_var_dict(room, name = "overtime")
#empty time of room r 
E = model.continuous_var_dict(room, name = "empty_time")

## Constraints

In [18]:
# each order positon can only be taken by one surgery, no matter in which room
for k in order:
    model.add_constraint(sum(sum(X[i,r,k] for i in surgeries) for r in room) == 1)

# each surgery can only be operated once in one room
for i in surgeries:
    model.add_constraint(sum(sum(X[i,r,k] for k in order) for r in room) == 1)

# if a surgery i doesn't take place in room r with order k, then the corresponding S[r,k] must be zero
# if a surgery i take place in room r with order k, then the corresponding surgery start time counts
# non-gegativity
for r in room:
    for k in order:
        model.add_constraint(S[r,k] <= sum(X[i,r,k]*bigM for i in surgeries))
        model.add_constraint(S[r,k] >= 0)

# let the first surgery be operated in room 1
model.add_constraint(sum(X[i,1,1] for i in surgeries) == 1)
model.add_constraint(S[1,1] - sum(X[i,1,1]*tp[inst[i-1]-1] for i in surgeries) >= 0)
model.add_constraint(S[2,1] == 0)

# the first surgery has no idle time and is operated directly after the preparation is done
# therefore the start time of first surgery equals to its corresponding preparation time
model.add_constraint(sum(S[r,1] for r in room) == sum(sum(X[i,r,1]*tp[inst[i-1]-1] for r in room) for i in surgeries))

# the surgery starts only after the last surgery is done
# therefore the start time of surgery is larger than the start time of last surgery plus its surgery time
for k in range(1,total_surgeries):
    model.add_constraint(sum(S[r,k] + sum(X[i,r,k]*ts[inst[i-1]-1] for i in surgeries) for r in room) <= 
                         sum(S[r,k+1] for r in room))

# idle time in room r with order k exists only when there is a surgery planned
# if there is no surgery, then the idle time is zero
# non-negativity
for r in room:
    for k in order:
        model.add_constraint(V[r,k] <= sum(X[i,r,k]*bigM for i in surgeries))
        model.add_constraint(V[r,k] >= 0)

# the first surgery has no idle time 
model.add_constraint(V[1,1] + V[2,1] == 0)

for r in room:
    # identify whether overtime in each room exists
    # if the time after clean-up of last surgery exceeds the shift length, then overtime exists and O[r] = 1, if not, then O[r] = 0
    model.add_constraint(O[r]*bigM 
                         >= sum(sum(X[i,r,k]*(tp[inst[i-1]-1] + ts[inst[i-1]-1] + tc[inst[i-1]-1]) for i in surgeries) for k in order) 
                             + sum(V[r,k] for k in order) - T*60)
    
    # relate empty time and overtime, empty time and overtime can't coexist in the same room
    model.add_constraint(E[r] <= (1-O[r])*bigM)
    
    # equation of the total used time in each room
    # for each room: time needed for all surgeries performed + all idle time + possible empty time = the shift length + possible overtime
    model.add_constraint(sum(sum(X[i,r,k]*(tp[inst[i-1]-1] + ts[inst[i-1]-1] + tc[inst[i-1]-1]) for i in surgeries) for k in order) 
                         + sum(V[r,k] for k in order) + E[r] == OT[r] + T*60)
    
    # relate the OT[r] and O[r], if overtime in one room doesn't exist, then OT[r] = 0
    model.add_constraint(OT[r] <= O[r]*bigM)
    
    # non-negativity
    model.add_constraint(OT[r] >= 0)
    model.add_constraint(E[r] >= 0)

# give idle time a lower bound
# the idle time of each surgery >= the start time of the surgery 
#                                 - its corresponding preparation time  - the used time of all surgeries ahead 
#                                 - all possible idle time of surgeries ahead
for r in room:
    for k in range(1,total_surgeries):
        model.add_constraint(V[r,k+1] >= S[r,k+1] - sum(X[i,r,k+1]*tp[inst[i-1]-1] for i in surgeries) 
                             - sum(sum(X[i,r,k1]*(tp[inst[i-1]-1] + ts[inst[i-1]-1] + tc[inst[i-1]-1]) for i in surgeries) for k1 in range(1,k+1))
                             - sum(V[r,k2] for k2 in range(1,k+1))
                            )
        
# consider all possible combinations in which two surgeries are performed consecutively in one room, no matter whether there is one or two 
# or three or more surgeries are performed between them in another room.

# if there are totally 5 surgeries, the combinations of two consecutively performed surgeries could be 1&2, 1&3, 1&4, 1&5, 2&3, 2&4, 2&5, 3&4, 3&5, 4&5
# if one of the combinations is ture (two surgeries are performed consecutively in one room), then LHS is the time after clean-up of a surgery 
# and the RHS is the start time of the following surgery - its corresponding preparation time - its possible idle time, which is exactly the time after clean-up of last surgery
# this also holds for combinations in which two surgeries are in the same room but not consecutively.

# if two surgeries of one combination is not in the same room, then the constraint will be either the time after clean-up of a surgery <= bigM
# or 0 <= a certain time + bigM. 
for r in room:
    for k in range(1,total_surgeries):
        for k1 in range(k,total_surgeries):
            model.add_constraint(S[r,k] + sum(X[i,r,k]*(ts[inst[i-1]-1] + tc[inst[i-1]-1]) for i in surgeries) <= 
                                 S[r,k1+1] - sum(X[i,r,k1+1]*tp[inst[i-1]-1] for i in surgeries) - V[r,k1+1] 
                                 + (1-sum(X[i,r,k1+1] for i in surgeries))*bigM
                                )
                                

## Objective Function

In [19]:
#surgeon waiting time
waiting_time = sum(S[r,k+1] - S[r,k] - sum(X[i,r,k]*ts[inst[i-1]-1] for i in surgeries) for r in room for k in range(1,total_surgeries))

#vacant time = idle time + empty time
vacant_time = sum(sum(V[r,k] for r in room) for k in order)

#overtime
overtime = sum(OT[r] for r in room)

#total_time = waiting_time + vacant_time + overtime
total_cost = cw/60*waiting_time + cv/60*vacant_time + co/60*overtime

model.minimize(total_cost)

## Solve the model

In [20]:
model.solve()
model.report()
model.print_solution()
print("The number of constraints is", model.number_of_constraints)
print("The CPU time is", model.solve_details.time, "seconds") # print the CPU time for the solve process

* model nightmare_worst solved with objective = 13500.884
objective: 13500.884
  surgery_start_time_1_1=13.000
  surgery_start_time_1_3=91.000
  surgery_start_time_1_5=347.200
  surgery_start_time_1_7=653.500
  surgery_start_time_2_2=32.400
  surgery_start_time_2_4=232.600
  surgery_start_time_2_6=490.400
  whether_surgery_1_1_1=1
  whether_surgery_2_2_6=1
  whether_surgery_3_1_3=1
  whether_surgery_4_1_7=1
  whether_surgery_5_2_4=1
  whether_surgery_6_1_5=1
  whether_surgery_7_2_2=1
  idle_time_1_5=30.100
  idle_time_1_7=12.900
  idle_time_2_2=1.800
  whether_overtime_1=1
  whether_overtime_2=1
  overtime_1=584.500
  overtime_2=265.300
The number of constraints is 147
The CPU time is 0.4850000001024455 seconds


## Show the optimal sequence and OR assignment

In [21]:
slist = instances.iloc[case,3:total_surgeries+3].tolist()

# show optimal sequence
res_order = []
for k in order:
    for i in surgeries:
        for r in room:
            if X[i,r,k].solution_value>0.5:
                res_order.append(slist[i-1])
                
# show OR assignment
res_order_1 = []
res_order_2 = []
for k in order:
    for i in surgeries:
        if X[i,1,k].solution_value>0.5:
            res_order_1.append(slist[i-1])
        if X[i,2,k].solution_value>0.5:
            res_order_2.append(slist[i-1])

print("The optimal sequence should be:",res_order)
print("OR1's schedule (in order) should be:",res_order_1)
print("OR2's schedule (in order) should be:",res_order_2)

The optimal sequence should be: ['A', 'J', 'D', 'G', 'G', 'B', 'E']
OR1's schedule (in order) should be: ['A', 'D', 'G', 'E']
OR2's schedule (in order) should be: ['J', 'G', 'B']


# Stochastic Model

In [22]:
model = Model(name = "nightmare_stochastic")

## Input Parameters

In [23]:
# Import historical data from excel
tp_scenarios = data.parse('Data_Robust_tp')
ts_scenarios = data.parse('Data_Robust_ts')
tc_scenarios = data.parse('Data_Robust_tc')

In [24]:
# add scenario relevent indicies and parameters
prob_s = 0.01
scenarios = list(range(1,101))

## Define Deicision Variables

In [25]:
#binary
X = model.binary_var_cube(surgeries, room, order, name = "whether_surgery")
#start time of surgery
S = model.continuous_var_cube(room, order,scenarios, name = "surgery_start_time")
#vacant time
V = model.continuous_var_cube(room, order,scenarios, name = "idle_time")
#binary, whether overtime at room r
O = model.binary_var_matrix(room, scenarios, name = "whether_overtime")
#overtime at room r
OT = model.continuous_var_matrix(room, scenarios, name = "overtime")
#empty time of room r 
E = model.continuous_var_matrix(room, scenarios, name = "empty_time")
#Total waiting time of each scenario
W = model.continuous_var_dict(scenarios, name = "waiting_time")
#Total cost for each scenario
C = model.continuous_var_dict(scenarios, name = "costs")

## Constraints

In [26]:
####### Independent constraints #######

# Only one surgery is in each position in each room
for k in order:
    model.add_constraint(sum(sum(X[i,r,k] for i in surgeries) for r in room) == 1)
# Only one position is asssigned to each surgery in each room    
for i in surgeries:
    model.add_constraint(sum(sum(X[i,r,k] for k in order) for r in room) == 1)
    
model.add_constraint(sum(X[i,1,1] for i in surgeries) == 1)
    

####### Dependent constraints #######

for s in scenarios:
    #Linking constraints, start time only exists when surgery is assigned to each position and room in each scenarios
    for r in room:
        for k in order:
            model.add_constraint(S[r,k,s] <= sum(X[i,r,k]*bigM for i in surgeries))
            model.add_constraint(S[r,k,s] >= 0)
    
    #The start time of the first surgery must be larger than the preparation time
    model.add_constraint(S[1,1,s] - sum(X[i,1,1]*tp_scenarios.iloc[s-1,inst[i-1]] for i in surgeries) >= 0)
    #No surgery is in the first position in the second room
    model.add_constraint(S[2,1,s] == 0)
    
    #Start time of the former surgery plus its surgery time is smaller than the start time of the next surgery cosidering both ORS.
    for k in range(1,total_surgeries):
        model.add_constraint(sum(S[r,k,s] + sum(X[i,r,k]*ts_scenarios.iloc[s-1,inst[i-1]]
                                                for i in surgeries) for r in room) <= sum(S[r,k+1,s] for r in room))
    
    #Start time of the first surgeries in both rooms is equal to the preparation time of the surgery
    model.add_constraint(sum(S[r,1,s] for r in room) == sum(X[i,r,1]*tp_scenarios.iloc[s-1,inst[i-1]] for i in surgeries  for r in room))
    
    #For surgeries in the same room,
    #Start time of former surgery plus the time of surgery and cleaning is smaller than or equal to the start time of the next surgery minus the preparation time 
    #there is idle time when smaller, no idle time when equal
    for r in room:
        for k in range(1,total_surgeries):
            for k1 in range(k,total_surgeries):
                model.add_constraint(S[r,k,s] + sum(X[i,r,k]*(ts_scenarios.iloc[s-1,inst[i-1]] + tc_scenarios.iloc[s-1,inst[i-1]]) for i in surgeries) <= 
                                 S[r,k1+1,s] - sum(X[i,r,k1+1]*tp_scenarios.iloc[s-1,inst[i-1]] for i in surgeries) - V[r,k1+1,s] 
                                 + (1-sum(X[i,r,k1+1] for i in surgeries))*bigM
                                )
    #There is no idle time when the surgery is not assigned to this position
    for r in room:
        for k in order:
            model.add_constraint(V[r,k,s] <= sum(X[i,r,k]*bigM for i in surgeries))
            model.add_constraint(V[r,k,s] >= 0)
    
    model.add_constraint(V[1,1,s] + V[2,1,s] == 0)
    
    for r in room:
        model.add_constraint(O[r,s]*bigM >= sum(sum(X[i,r,k]*(tp_scenarios.iloc[s-1,inst[i-1]] +
                                                            ts_scenarios.iloc[s-1,inst[i-1]] + 
                                                            tc_scenarios.iloc[s-1,inst[i-1]]) 
                                                  for i in surgeries) for k in order) + 
                             sum(V[r,k,s] for k in order)- T*60)
    
        model.add_constraint(E[r,s] <= (1-O[r,s])*bigM)
    
        model.add_constraint(sum(sum(X[i,r,k]*(tp_scenarios.iloc[s-1,inst[i-1]] + ts_scenarios.iloc[s-1,inst[i-1]] + tc_scenarios.iloc[s-1,inst[i-1]]) for i in surgeries) for k in order) 
                         + sum(V[r,k,s] for k in order) + E[r,s] == OT[r,s] + T*60)
    
        model.add_constraint(OT[r,s] <= O[r,s]*bigM)
    
        model.add_constraint(OT[r,s] >= 0)
        model.add_constraint(E[r,s] >= 0)
    
    for r in room:
        for k in range(1,total_surgeries):
            model.add_constraint(V[r,k+1,s] >= S[r,k+1,s] - sum(X[i,r,k+1]*tp_scenarios.iloc[s-1,inst[i-1]] for i in surgeries) 
                             - sum(sum(X[i,r,k1]*(tp_scenarios.iloc[s-1,inst[i-1]] + ts_scenarios.iloc[s-1,inst[i-1]] + tc_scenarios.iloc[s-1,inst[i-1]]) for i in surgeries) for k1 in range(1,k+1))
                             - sum(V[r,k2,s] for k2 in range(1,k+1))
                            )
    
    model.add_constraint(W[s]==sum(S[r,k+1,s] - S[r,k,s] - sum(X[i,r,k]*ts_scenarios.iloc[s-1,inst[i-1]] for i in surgeries) for r in room for k in range(1,total_surgeries)))
    model.add_constraint(C[s]==cw/60*W[s] + cv/60*sum(sum(V[r,k,s] for r in room) for k in order) + co/60*sum(OT[r,s] for r in room))
    

## Objective Function

In [27]:
total_cost = sum(prob_s * C[s] for s in scenarios)

model.minimize(total_cost)

## Solve the model

In [28]:
model.solve()
model.report()
model.print_solution()
print("The number of constraints is", model.number_of_constraints)
print("The CPU time is", model.solve_details.time, "seconds") # print the CPU time for the solve process

* model nightmare_stochastic solved with objective = 2868.812
objective: 2868.812
  whether_surgery_1_1_1=1
  whether_surgery_2_2_6=1
  whether_surgery_3_2_2=1
  whether_surgery_4_1_7=1
  whether_surgery_5_1_5=1
  whether_surgery_6_2_4=1
  whether_surgery_7_1_3=1
  surgery_start_time_1_1_1=11.800
  surgery_start_time_1_1_2=2.200
  surgery_start_time_1_1_3=4.100
  surgery_start_time_1_1_4=4.600
  surgery_start_time_1_1_5=6.200
  surgery_start_time_1_1_6=2.100
  surgery_start_time_1_1_7=6.700
  surgery_start_time_1_1_8=4.800
  surgery_start_time_1_1_9=4.300
  surgery_start_time_1_1_10=4.300
  surgery_start_time_1_1_11=4.800
  surgery_start_time_1_1_12=3.600
  surgery_start_time_1_1_13=5.600
  surgery_start_time_1_1_14=1.800
  surgery_start_time_1_1_15=2.900
  surgery_start_time_1_1_16=3.100
  surgery_start_time_1_1_17=3.300
  surgery_start_time_1_1_18=3.800
  surgery_start_time_1_1_19=5.000
  surgery_start_time_1_1_20=3.800
  surgery_start_time_1_1_21=11.100
  surgery_start_time_1_1_22=6

## Show the optimal sequence and OR assignment

In [29]:
slist = instances.iloc[case,3:total_surgeries+3].tolist()

# show optimal sequence
res_order = []
for k in order:
    for i in surgeries:
        for r in room:
            if X[i,r,k].solution_value>0.5:
                res_order.append(slist[i-1])
                
# show OR assignment
res_order_1 = []
res_order_2 = []
for k in order:
    for i in surgeries:
        if X[i,1,k].solution_value>0.5:
            res_order_1.append(slist[i-1])
        if X[i,2,k].solution_value>0.5:
            res_order_2.append(slist[i-1])

print("The optimal sequence should be:",res_order)
print("OR1's schedule (in order) should be:",res_order_1)
print("OR2's schedule (in order) should be:",res_order_2)

The optimal sequence should be: ['A', 'D', 'J', 'G', 'G', 'B', 'E']
OR1's schedule (in order) should be: ['A', 'J', 'G', 'E']
OR2's schedule (in order) should be: ['D', 'G', 'B']
